In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load sales and prices datasets, both without nulls and duplicated
df_sales = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Nuclio/datasets/TFM/df_sales_resample.csv')
df_price = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Nuclio/datasets/TFM/df_price_merge.csv')